## Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import seaborn as sns
import pandas as pd
import pickle
import scanpy as sc
sc.set_figure_params(dpi=100, dpi_save=300)
import scvi
import anndata as ad
from matplotlib import pyplot, cm
import os
from math import ceil
from scipy.stats import spearmanr
import math

import leidenalg

from anndata import AnnData
import scanpy as sc
from scanpy import read
import pandas as pd
from sciPENN.Preprocessing import preprocess

import matplotlib.pyplot as plt
print(scvi.__version__)

0.9.1


# Read data: Pbmc (train), H1N1 (test)

In [2]:
adata_gene = sc.read("../Data/pbmc/pbmc_gene.h5ad")
adata_protein = sc.read("../Data/pbmc/pbmc_protein.h5ad")

In [3]:
adata_malt_gene = sc.read_10x_h5("../Data/malt_10k_protein_v3_filtered_feature_bc_matrix.h5")
adata_malt = sc.read("../Data/filtered_feature_bc_matrix/matrix.mtx").T
malt_features =  pd.read_csv("../Data/filtered_feature_bc_matrix/features.tsv", sep="\t", header=None)

adata_malt.var["feature_type"] = list(malt_features[2])
adata_malt.obs_names = adata_malt_gene.obs_names
adata_malt.var['protein_names'] = list(malt_features[0])
adata_malt.var_names = list(malt_features[0])

adata_malt_protein = adata_malt[:,adata_malt.var['feature_type'] == 'Antibody Capture']

adata_malt_gene.var_names_make_unique()

adata_gene_test = adata_malt_gene.copy()

adata_protein_test = adata_malt_protein.copy()
adata_gene_test.obs['sample'] = [1]*8412
adata_protein_test.obs['sample'] = [1]*8412
adata_protein_test

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


AnnData object with n_obs × n_vars = 8412 × 17
    obs: 'sample'
    var: 'feature_type', 'protein_names'

In [4]:
common_genes = np.intersect1d(adata_gene.var.index, adata_gene_test.var.index)
common_proteins = np.intersect1d(adata_protein.var.index, adata_protein_test.var.index)

In [5]:
adata_malt_protein

View of AnnData object with n_obs × n_vars = 8412 × 17
    var: 'feature_type', 'protein_names'

# Selecting highly variable genes - using gene expression measures from test data 

In [7]:
gene_train, protein_train, gene_test, bools, train_keys, categories = preprocess([adata_gene], [adata_protein], adata_gene_test, train_batchkeys = ["donor"], gene_list = [], select_hvg = True, cell_normalize = True, log_normalize = True, gene_normalize = True, min_cells = 30, min_genes = 200)


QC Filtering Training Cells
QC Filtering Testing Cells

QC Filtering Training Genes
QC Filtering Testing Genes

Normalizing Training Cells
Normalizing Testing Cells

Log-Normalizing Training Data
Log-Normalizing Testing Data

Finding HVGs


... storing 'orig.ident' as categorical
... storing 'lane' as categorical
... storing 'donor' as categorical
... storing 'time' as categorical
... storing 'celltype.l1' as categorical
... storing 'celltype.l2' as categorical
... storing 'celltype.l3' as categorical
... storing 'Phase' as categorical
... storing 'batch' as categorical
... storing 'Dataset' as categorical
... storing 'feature_types-1' as categorical
... storing 'genome-1' as categorical
/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)



Normalizing Gene Training Data by Batch


100%|██████████| 8/8 [00:06<00:00,  1.20it/s]



Normalizing Protein Training Data by Batch


100%|██████████| 8/8 [00:02<00:00,  2.96it/s]



Normalizing Gene Testing Data by Batch


100%|██████████| 1/1 [00:00<00:00,  6.25it/s]


In [8]:
hvg = gene_test.var_names
cells_test = gene_test.obs_names
cells_train = gene_train.obs_names

In [9]:
del gene_train
del gene_test
del protein_train

import gc
gc.collect()

3302

# Format data

In [10]:
# What proteins overlap between the test and train data?

train_protein = adata_protein.var_names
test_protein = adata_protein_test.var_names
overlap_protein = train_protein[train_protein.isin(test_protein)]

In [11]:
## Subsetting the data by the HVG - pbmc

adata_gene_pbmc_hvg = adata_gene[cells_train, hvg].copy()

In [12]:

## Subsetting the data by the HVG - h1n1

adata_gene_h1n1_hvg = adata_gene_test[cells_test, hvg].copy()

In [13]:
(adata_gene_pbmc_hvg.var.index == adata_gene_h1n1_hvg.var.index).mean()

1.0

In [14]:
adata_protein = adata_protein[cells_train, :].copy()
adata_protein_test = adata_protein_test[cells_test, :].copy()

In [15]:
# Batches (subject) in training data - pbmc (8 subjects)

adata_gene_pbmc_hvg.obs['patient'] = pd.DataFrame(adata_gene_pbmc_hvg.obs['donor']).copy()
adata_gene_pbmc_hvg.obs['patient'] = adata_gene_pbmc_hvg.obs['donor'].astype("str")

In [16]:
# Batches (subject) in test data - h1n1 (20 subjects)

adata_gene_h1n1_hvg.obs['patient'] = pd.DataFrame(adata_gene_h1n1_hvg.obs['sample']).copy()
adata_gene_h1n1_hvg.obs['patient'] = adata_gene_h1n1_hvg.obs['patient'].astype("str")

In [17]:
## Combine data

adata = ad.concat([adata_gene_pbmc_hvg.copy(), adata_gene_h1n1_hvg.copy()],
                     join='outer')

### Note: Train on PBMC

In [18]:
train_patients = adata.obs["patient"].unique()[0:8]

In [19]:
test_patients = adata.obs["patient"].unique()[8:]

# Subset data based on HVGs and Hold Out Test Protein Set

In [20]:
adata_final = adata.copy()

In [21]:
held_out_proteins = adata_protein_test[cells_test, overlap_protein].copy()

### Now we hold-out the proteins for the test patients dataset. To do so, we can replace all the values with 0s. We will store the original values to validate after training.

In [22]:
# Modified this code cell to predict all p = 224 proteins

n, p = adata_protein.shape
n_H1N1, p_H1N1 = adata_protein_test.shape

protein_dat = pd.DataFrame(np.zeros(shape = (n + n_H1N1, p), dtype = 'float32'), 
                           index = list(adata_protein.obs_names) + list(adata_protein_test.obs_names),
                           columns = adata_protein.var_names)

protein_dat.iloc[:n] = adata_protein.X.toarray().copy() #fill the protein training data, leave test data as 0s

adata_final.obsm["protein_expression"] = protein_dat


# Remove additional data from memory:

In [23]:
del adata_gene
del adata_protein
del adata_protein_test
del adata_gene_test
del adata_gene_pbmc_hvg
del adata

# Run TotalVI

In [24]:
scvi.data.setup_anndata(adata_final, batch_key="patient", 
                        protein_expression_obsm_key="protein_expression")

INFO     Using batches from adata.obs["patient"]                                             
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Using protein expression from adata.obsm['protein_expression']                      
INFO     Using protein names from columns of adata.obsm['protein_expression']                
INFO     Found batches with missing protein expression                                       
INFO     Successfully registered anndata object containing 170133 cells, 1000 vars, 9        
         batches, 1 labels, and 224 proteins. Also registered 0 extra categorical covariates 
         and 0 extra continuous covariates.                                                  
INFO     Please do not further modify adata until model is t

In [25]:
scvi.data.view_anndata_setup(adata_final)

Anndata setup with scvi-tools version 0.9.1.

              Data Summary               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃             Data             ┃ Count  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│            Cells             │ 170133 │
│             Vars             │  1000  │
│            Labels            │   1    │
│           Batches            │   9    │
│           Proteins           │  224   │
│ Extra Categorical Covariates │   0    │
│ Extra Continuous Covariates  │   0    │
└──────────────────────────────┴────────┘

                   SCVI Data Registry                    
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Data        ┃       scvi-tools Location        ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         X          │             adata.X              │
│   batch_indices    │     adata.obs['_scvi_batch']     │
│    local_l_mean    │ adata.obs['_scvi_local_l_mean']  │
│    local_l_var     │  adata.obs['_scvi_local_l_var']  │
│       labels       │    adata.obs['_scvi_labels']     │
│ protein_expression │ adata.obsm['protein_expression'] │
└────────────────────┴──────────────────────────────────┘

                        Label Categories                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                     Batch Categories                      
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['patient'] │     1      │          0          │
│                      │     P1     │          1          │
│                      │     P2     │          2          │
│                      │     P3     │          3          │
│                      │     P4     │          4          │
│                      │     P5     │          5          │
│                      │     P6     │          6          │
│                      │     P7     │          7          │
│                      │     P8     │          8          │
└──────────────────────┴────────────┴─────────────────────┘

In [26]:
totalvae = scvi.model.TOTALVI(
    adata_final,
    latent_distribution = "normal",
    n_layers_decoder = 2)

In [27]:
# Training with the default number of epochs 

# Training with the default number of epochs 
n_epochs = 400
lr = 4e-3

if os.path.isdir('weights_dir/totalvi_seurattomalt'):
    totalvae = totalvae.load("weights_dir/totalvi_seurattomalt", adata = adata_final)
else:
    totalvae.train(max_epochs=400)
    plt.plot(totalvae.history["elbo_validation"], label="test")
    plt.title("Negative ELBO over training epochs")
    plt.legend()
    
    totalvae.save("weights_dir/totalvi_seurattomalt")

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


INFO     Found batches with missing protein expression                                       
INFO     Using data from adata.X                                                             
INFO     Computing library size prior per batch                                              
INFO     Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels',     
         'protein_expression']                                                               
INFO     Successfully registered anndata object containing 170133 cells, 1000 vars, 9        
         batches, 1 labels, and 224 proteins. Also registered 0 extra categorical covariates 
         and 0 extra continuous covariates.                                                  


# Analyze output - Results on training data

In [28]:
_, protein_means = totalvae.get_normalized_expression(
    transform_batch=train_patients,
    include_protein_background=True,
    sample_protein_mixing=False,
    return_mean=True,
)

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Note that: transform_batch is a power parameter. Setting this allows one to predict the expression of cells as if they came from the inputted batch. In this case, we’ve observed protein expression in the training batchs “RPM211 and RPM232” (batch categories from original adata object), but we have no protein expression in the test batchs “RPM215 and RPM218”. We’d like to take the cells of the trainig batch and make a counterfactual prediction: “What would the expression look like if my batch "RPM211 and RPM232" cells came from batch “RPM215 and RPM218”?”

In [29]:
X_totalVI = pd.DataFrame(totalvae.get_latent_representation(), index = adata_final.obs.index)
X_totalVI.to_csv("totalvi_maltembedding.csv")

# Imputed protein expression: 

In [30]:
true_protein_test = pd.DataFrame(held_out_proteins.X.toarray(), index = held_out_proteins.obs.index, columns = held_out_proteins.var.index)

In [31]:
imputed_pros = protein_means[adata_final.obs.patient.isin(test_patients)]

pat_names = adata_final.obs['patient'].isin(test_patients)
patients = adata_final.obs.patient[pat_names].values
imputed_proteins_test = imputed_pros.copy()
# imputed_proteins_test = imputed_pros[overlap_protein] # Subset totalvi output to only include overlapping proteins

In [32]:
def corr2_coeff(A, B, pearson = True):
    if pearson:
        # Rowwise mean of input arrays & subtract from input arrays themeselves
        A_mA = A - A.mean(1)[:, None]
        B_mB = B - B.mean(1)[:, None]

        # Sum of squares across rows
        ssA = (A_mA**2).sum(1)
        ssB = (B_mB**2).sum(1)

        # Finally get corr coeff
        corr_mat = np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None],ssB[None]))
        
        return corr_mat[range(corr_mat.shape[0]), range(corr_mat.shape[0])]
    
    else:
        corrs = [0.] * A.shape[0]
        
        for i in range(A.shape[0]):
            corrs[i] = spearmanr(A[i], B[i])[0]
            
        return corrs

In [33]:
# Normalize totalvi output, and gold standard counts

true_protein_test = AnnData(true_protein_test)
imputed_proteins_test = AnnData(imputed_proteins_test)

sc.pp.normalize_total(true_protein_test)
sc.pp.log1p(true_protein_test)

sc.pp.normalize_total(imputed_proteins_test)
sc.pp.log1p(imputed_proteins_test)

for patient in test_patients:
    indices = [x == patient for x in patients]
    sub_adata = imputed_proteins_test[indices]
    sc.pp.scale(sub_adata)
    imputed_proteins_test[indices] = sub_adata.X
    
    sub_adata = true_protein_test[indices]
    sc.pp.scale(sub_adata)
    true_protein_test[indices] = sub_adata.X

true_protein_test = pd.DataFrame(true_protein_test.X, columns = true_protein_test.var.index)
imputed_proteins_test = pd.DataFrame(imputed_proteins_test[:, overlap_protein].X, columns = overlap_protein)

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/scanpy/preprocessing/_simple.py:810: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [34]:
sq = lambda x, y: (x - y)**2

In [35]:
corrs_table = np.zeros((imputed_proteins_test.shape[1], len(np.unique(patients))))
sq_table = corrs_table.copy()

for i, patient in enumerate(np.unique(patients)):
    truth = true_protein_test[patients == patient].to_numpy()
    imputed = imputed_proteins_test[patients == patient].to_numpy()

    corrs_table[:, i] = corr2_coeff(truth.T, imputed.T)
    sq_table[:, i] = sq(truth, imputed).mean(axis = 0)

if np.isnan(corrs_table).sum() > 0:
    corrs_table[np.isnan(corrs_table)] = 0
    
corrs_table = pd.DataFrame(corrs_table)
sq_table = pd.DataFrame(sq_table)
corrs_table.index, corrs_table.columns = imputed_proteins_test.columns, np.unique(patients)
sq_table.index, sq_table.columns = imputed_proteins_test.columns, np.unique(patients)

In [36]:
#here are correlations

corrs_table

,1
index,
CD19,0.857451
CD45RA,0.542695
CD8a,0.690626
CD14,-0.103021
CD25,0.461812
CD45RO,0.674743
TIGIT,0.455002
CD127,0.786754
CD15,0.108171


In [37]:
#here are correlations

corrs_table.mean().mean()

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.4601380318403244

In [38]:
corrs_table.to_csv('corrs_results/totalvi_malt.csv')

In [39]:
sq_table

,1
index,
CD19,0.285065
CD45RA,0.914502
CD8a,0.618674
CD14,2.205783
CD25,1.076250
CD45RO,0.650437
TIGIT,1.089867
CD127,0.426439
CD15,1.783443


In [40]:
sq_table.mean().mean()

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1.07959543466568

In [41]:
sq_table.to_csv('mse_results/totalvi_malt.csv')

In [42]:
imputed_proteins_test.to_csv('totalvi_maltfeatures.csv')

In [43]:
_, protein_means_samples = totalvae.get_normalized_expression(
    transform_batch=train_patients,
    n_samples=1000,
    include_protein_background=True,
    sample_protein_mixing=False,
    return_mean=False,
    indices = list(range(n, len(protein_dat)))
)

In [44]:
name_map = {protein: index for index, protein in enumerate(imputed_pros.columns)}
index_overlap = [name_map[protein] for protein in overlap_protein]

In [45]:
protein_means_samples_norm = np.empty(protein_means_samples.shape)

imputed_pros = protein_means[adata_final.obs.patient.isin(test_patients)]

# normalize totalvi sample output
sf = protein_means_samples.sum(axis = 1)
sf = np.median(sf, axis = 0)[None, :]/sf

protein_means_samples_norm = protein_means_samples * sf[:, None, :]
protein_means_samples_norm = np.log(protein_means_samples_norm + 1)
protein_means_samples_norm = protein_means_samples_norm[:, index_overlap]

for patient in test_patients:
    indices = [x == patient for x in patients]
    sub_data = protein_means_samples_norm[indices]
    mean, sd = sub_data.mean(axis = 0), sub_data.std(axis = 0)
    sub_data = (sub_data - mean)/sd
    protein_means_samples_norm[indices] = sub_data

In [46]:
protein_means_samples_norm.shape

(8385, 10, 1000)

In [47]:
q10 = np.percentile(protein_means_samples_norm, 10, axis = 2)
q25 = np.percentile(protein_means_samples_norm, 25, axis = 2)
q75 = np.percentile(protein_means_samples_norm, 75, axis = 2)
q90 = np.percentile(protein_means_samples_norm, 90, axis = 2)

In [48]:
q90.shape

(8385, 10)

In [49]:
cols = imputed_pros[overlap_protein].columns
names = imputed_pros[overlap_protein].index
q10 = pd.DataFrame(q10,  columns=cols)
q90 = pd.DataFrame(q90,  columns=cols)
q25 = pd.DataFrame(q25,  columns=cols)
q75 = pd.DataFrame(q75, columns=cols)

In [51]:
base_path = "totalVI_quantiles_malt"

In [52]:
true_protein_test.to_csv(os.path.join(base_path, "truth.csv"))

In [53]:
r50 = (true_protein_test < q75)
l50 = (true_protein_test > q25)

print(f"Effective Coverage Probability for Nominal 50% PIs: {(r50*l50).mean()}")
print(f"Mean effective Coverage Probability for Nominal 50% PI: {(r50*l50).mean().mean()}")

Effective Coverage Probability for Nominal 50% PIs: CD19      0.081216
CD45RA    0.061419
CD8a      0.129875
CD14      0.056530
CD25      0.100894
CD45RO    0.128801
TIGIT     0.103995
CD127     0.116637
CD15      0.125343
CD16      0.075492
dtype: float64
Mean effective Coverage Probability for Nominal 50% PI: 0.09802027429934405


/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [54]:
r80 = (true_protein_test < q90)
l80 = (true_protein_test > q10)

print(f"Effective Coverage Probability for Nominal 80% PIs: {(r80*l80).mean()}")
print(f"Mean effective Coverage Probability for Nominal 80% PI: {(r80*l80).mean().mean()}")

Effective Coverage Probability for Nominal 80% PIs: CD19      0.158259
CD45RA    0.114013
CD8a      0.240429
CD14      0.108289
CD25      0.186166
CD45RO    0.236732
TIGIT     0.191413
CD127     0.220155
CD15      0.229577
CD16      0.146571
dtype: float64
Mean effective Coverage Probability for Nominal 80% PI: 0.18316040548598686


/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/pandas/core/computation/expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


In [55]:
base_path = 'totalVI_quantiles_malt'

if not os.path.isdir(base_path):
    os.mkdir(base_path)

/home/jlakkis/miniconda3/envs/scipen/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
q10.to_csv(os.path.join(base_path, 'q10.csv'))
q25.to_csv(os.path.join(base_path, 'q25.csv'))
q75.to_csv(os.path.join(base_path, 'q75.csv'))
q90.to_csv(os.path.join(base_path, 'q90.csv'))